In [1]:
import numpy as np
import sys
import time

In [4]:
def inputfile(filename):
    try:
        f = open(filename, 'r')
    except IOError:
        print ("Cannot open file %s\n" % filename)
        sys.exit("bye")

    # read data
    data = f.readlines()
    f.close()

    line0 = data[0].split()
    #print line0

    if len(line0) == 0:
        sys.exit("empty first line")

    n = int(line0[1])
    print ('n = ', n)

    matrix = np.zeros((n,n))

    line1 = data[1].split()
    #should check that line1[0] is the string 'matrix'
    for i in range(n):
        #read line i + 2
        theline = data[i+2].split()
        #print i, " -> ", theline
        for j in range(n):
            valueij = float(theline[j])
            #print i, j, numberij
            matrix[i][j] = valueij

    return matrix, n

def runpower(M, n, eig, v):
    #Params
    oldnormw = 0
    tolerance = 1e-06
    convergence = False

    T = 1 #number of iterations
    for t in range(T):
        if t == 0:
            w0 = np.random.rand(n).reshape(-1,1)
            #w0 = M@w/np.linalg.norm(M@w)
            #print(w0[0:10,:])
            #print(M[0:10,0:10])
            print('M1',M[0:5,0:5])
            M = M - eig*v@v.reshape(-1,1)
            print('M2',M[0:5,0:5])
            w = w0-((v@w0)*(v).reshape(-1,1))
            normw = np.linalg.norm(M@w)
            w = M@w/normw
            #print(normw)
            #print(w[0:10])
        else:
            normw = np.linalg.norm(M@w)
            #print(normw)
            w = M@w/normw

        #print('t: ',t,'normw: ',normw)

        if np.abs(normw - oldnormw)/normw < tolerance:
            convergence = True
            #print (t,'breaking')
            break
        oldnormw = normw
    #if convergence == False:
    #    raise error("Did not converge.")

    #calculate eigen using Raleigh quotient
    eig = (w.T@M@w)/(w.T@w)
    #print(eig)
    return w.T,eig

def eigen(M, n, tol):
    print('i=0')
    vector = np.zeros((M.shape[1],100000))
    eig = np.zeros(100000)
    vector[:,0], eig[0] = runpower(M, n, eig[0], vector[:,0])
    i = 0
    while (eig[i]/eig[0] >= tol):
        print('i=',i)
        if i == 1:
            break
        vector[:,i+1], eig[i+1] = runpower(M, n, eig[i], vector[:,i])
        i = i+1

    return vector, eig

In [27]:
M, n = inputfile('russell_cov.txt')
#print('M',M[0:10,0:10])
vector, eig = eigen(M,n,0.7)
#print(eig)

n =  947
i=0
M1 [[  2.13272989e-04   2.02451953e-04   4.66137539e-05   6.70634730e-05
    6.64694473e-05]
 [  2.02451953e-04   2.01184274e-04   4.09357886e-05   6.56836188e-05
    6.79113752e-05]
 [  4.66137539e-05   4.09357886e-05   1.15274037e-03   8.89513817e-05
    1.09235574e-04]
 [  6.70634730e-05   6.56836188e-05   8.89513817e-05   1.12789216e-04
    4.91404127e-05]
 [  6.64694473e-05   6.79113752e-05   1.09235574e-04   4.91404127e-05
    4.74213486e-04]]
M2 [[  2.13272989e-04   2.02451953e-04   4.66137539e-05   6.70634730e-05
    6.64694473e-05]
 [  2.02451953e-04   2.01184274e-04   4.09357886e-05   6.56836188e-05
    6.79113752e-05]
 [  4.66137539e-05   4.09357886e-05   1.15274037e-03   8.89513817e-05
    1.09235574e-04]
 [  6.70634730e-05   6.56836188e-05   8.89513817e-05   1.12789216e-04
    4.91404127e-05]
 [  6.64694473e-05   6.79113752e-05   1.09235574e-04   4.91404127e-05
    4.74213486e-04]]
i= 0
M1 [[  2.13272989e-04   2.02451953e-04   4.66137539e-05   6.70634730e-05
 

In [75]:
eig,eigv = np.linalg.eigh(M)
#print(eig)
#print(eigv)
print(eig[::-1][0:10])
print(eigv[:,937:946])
#seig = np.sort(eig)[::-1]
#print(seig[0:20])

[ 0.08317999  0.02177345  0.01899317  0.01812049  0.00789068  0.00628908
  0.00431285  0.00379286  0.00371762  0.00312842]
[[ 0.00912548 -0.00926399  0.00103688 ..., -0.01455155  0.0199985
  -0.0016435 ]
 [ 0.01283679 -0.01321723 -0.00200524 ..., -0.01784184  0.01869966
  -0.00156002]
 [-0.03438458  0.01325108  0.04129705 ...,  0.01444323 -0.02474624
   0.02363519]
 ..., 
 [-0.00604122  0.00716188 -0.00920907 ..., -0.00128186  0.00826078
   0.00581195]
 [ 0.03246925 -0.02456331  0.01625861 ..., -0.00499791  0.01337852
  -0.01800944]
 [ 0.01592957  0.01690769 -0.01307427 ..., -0.00776056  0.0035893
  -0.01237918]]


In [17]:
def runpower1(matrix, n):
    v = np.zeros(n)
    w = np.zeros(n)

    for j in range(n):
        v[j] = np.random.uniform(0,1)

    #print 'matrix', matrix
    #print 'v', v
    T = 10000 #number of iterations
    tolerance = 1e-06
    oldnormw = 0
    for t in range(T):
        w = matrix.dot(v)
        #print 't', t, 'w',w
        normw = (np.inner(w,w))**.5

        v = w/normw
        #print 't',t,'v',v

        #print 't',t,'normw',normw, 'old', oldnormw
        if np.abs(normw - oldnormw)/normw < tolerance:
            #print ' breaking'
            break
        oldnormw = normw
    #comment: if t reaches T-1 the algorithm has not converged to tolerance
    # within T iterations.  The function should return an error code in that
    # case
    eig = (v.T@M@v)/(v.T@v)
    return eig, v


In [60]:
def runpower2(matrix, n,v):
    #print(v)
    #print 'matrix', matrix
    #print 'v', v
    T = 10000 #number of iterations
    tolerance = 1e-06
    oldnormw = 0
    for t in range(T):
        w = matrix.dot(v)
        #print(w)
        #print 't', t, 'w',w
        normw = np.linalg.norm(w)

        v = w/normw
        #print 't',t,'v',v

        #print 't',t,'normw',normw, 'old', oldnormw
        if np.abs(normw - oldnormw)/normw < tolerance:
            #print ' breaking'
            break
        oldnormw = normw
    #comment: if t reaches T-1 the algorithm has not converged to tolerance
    # within T iterations.  The function should return an error code in that
    # case
    eig = (v.T@M@v)/(v.T@v)
    return eig, v

In [67]:
M, n = inputfile('russell_cov.txt')
eig, eigv = runpower1(M,n)

n =  947


In [68]:
M1= M - eig*eigv.reshape(-1,1)@np.array([eigv])
v = np.zeros(n)
w = np.zeros(n)

for j in range(n):
    v[j] = np.random.uniform(0,1)
w=v
#w = M@v/np.linalg.norm(M@v)
w0 = w.reshape(-1,1) - np.array([eigv])@w.reshape(-1,1)*eigv.reshape(-1,1)
eig2, eigv2 = runpower2(M1,n, w0)

In [71]:
vector = np.zeros((M.shape[1],100000))
e = np.zeros(100000)

In [74]:
vector[:,0] = np.ravel(eigv2)